<a href="https://colab.research.google.com/github/worklifesg/Python-for-Computer-Vision-with-OpenCV-and-Deep-Learning/blob/main/6.%20Object%20Tracking/1_ObjectTracking_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Object Tracking

In this section, we will cover:

- Basic Object Tracking Techniques
 - Optical Flow
 - Meanshift and Camshift
- Advanced Tracking
 - Built-In Tracking APIs

#### Optical Flow

- It is defined ss the pattern of apparent motion of image objects between 2 consecutive frames caused by the movement of object or camera.
- <b> Assumptions: </b>
 - Pixel intensities of object between consecutive frames <b> DONOT CHANGWE </b>
 - Neighbouring pixels have similar motion
- Methods:
 - Take given set of points and a frame.
 - Attempt to find those points in next frame.
 - Upto user to supply the points to track.
- We need to also see that in which direction object is moving or is it the camera which is moving.
-<b> General way to track a object: </b> We will pass <b> previous frame, previous points and current points </b> to the <b> Lucas Kanade </b> function.
- The function then attempts to locate points in the current frame.
- <b> Lucas Kanade </b> computes optical flow for a <b> sparse </b> feature set i.e. only points that it was told to track.
 - But to track all points in the video, we have to use <b> Gunner Farneback's algorithm that computes dense optical flow </b>. It will color them black if no flow is detected.

- <b> Optical Flow Equation:</b>
 - Consider an object with intensity $I (x, y, t)$, after time $dt$, it moves to by $dx$ and $dy$, now, the new intensity would be, $I (x+dx, y+dy, t+dt)$.
 - We, assume that the pixel intensities are constant between the two frames, i.e.,
\begin{align*}
 I (x, y, t) = I (x+dx, y+dy, t+dt)
\end{align*}

 - Then take taylor series approximation of right-hand side, resulting in,

 \begin{align*}
 \dfrac{dI}{dx}\delta x+\dfrac{dI}{dy}\delta y+\dfrac{dI}{dt}\delta t=0
 \end{align*}

 - Dividing equation above by $\delta t $ gives us <b> Optical Flow Equation:</b>

 \begin{align*}
 \dfrac{dI}{dx}u+\dfrac{dI}{dy}v+\dfrac{dI}{dt}=0
 \end{align*}

 - where $u= \delta x/\delta t, v= \delta y/\delta t$ and $dl/dx$ is image gradient along horizontal axis and $dl/dy$ is image gradient along vertical axis and $dl/dt$ is along the time. 



---


#### Lucas - Kanade Function

In [1]:
import cv2
import numpy as np


In [2]:
#Creating a dictionary to track corners
corner_track_params = dict(maxCorners=10,
                           qualityLevel=0.3,
                           minDistance=7,
                           blockSize=7)
#parameters for Lucas Kanade function
#Smaller window - more senstive to noise and may miss larger motions
'''
LK method using the idea of image pyramid 
Level0 - original image, Level1 - 1/2 resol, Level2 - 1/4 resol, Level3 - 1/8 resol, Level4 - 1/15 resol
At each level the image is blurred and subsample i.e. allows to find optical flow at various resolutions
'''
lk_params = dict(winSize=(200,200),
                 maxLevel=2,
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [ ]:
# live streaming capturing of Sparse Optical Flow

cap = cv2.VideoCapture(0)
ret,prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

# points to track

prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

#mask
mask = np.zeros_like(prev_frame)

while True:

  ret, frame = cap.read()
  frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

  nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts, None,**lk_params)

  good_new = nextPts[status==1]
  good_prev = prevPts[status==1]

  for i, (new,prev), in enumerate(zip(good_new,good_prev)):
    x_new,y_new = new.ravel()
    x_prev,y_prev = prev.ravel()

    mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)

    frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1) #draing circle on current points in a frame are
  
  img = cv2.add(frame,mask)
  cv2.imshow('tracking',img)

  k = cv2.waitKey(30) & 0xFF
  if k == 27:
    break
  
  prev_gray = frame_gray.copy()
  prevPts = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()